# Trabajar con datos

In [1]:
! pip show azure-ai-ml

## Conectar a workspace

In [2]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


Found the config file in: ./config.json


Conectado al Workspace: naturgyml4


## Listar los almacenes de datos

In [5]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_training_data
azureml_globaldatasets
workspacefilestore
workspaceartifactstore
workspaceworkingdirectory
workspaceblobstore


## Crear un datastore

To create a datastore and connect to a (already existing) storage, you'll need to specify:

- La clase para indicar con qué tipo de servicio de almacenamiento desea conectar.El siguiente ejemplo se conecta a un almacenamiento Blob (Azure Blob DataStore).
- name: El nombre del almacén de datos en el espacio de trabajo de Azure Machine Learning.
- description: Descripción opcional para proporcionar más información sobre el almacén de datos.
- account_name: El nombre de la cuenta de almacenamiento de Azure.
- container_name: El nombre del contenedor para almacenar blobs en la cuenta de almacenamiento de Azure.
- credentials: Proporcione el método de autenticación y las credenciales para autenticar.El siguiente ejemplo usa una clave de cuenta.

Reemplazar XXXX-XXXX con la clave de cuenta del storage account.

In [15]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

store = AzureBlobDatastore(
    name="blob_training_data",
    description="Blob Storage for training data",
    account_name="ntgysapc",
    container_name="training-data", 
    credentials=AccountKeyConfiguration(
        account_key="ZNXb7EFz/RF2j5LxHrSlW/sul50Jldw+gWqLJbyokzNKlb0UrQxjKGk2U6aroq87fOK40BYuaiPs+ASt22gd5A=="
    ),
)

ml_client.create_or_update(store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'blob_training_data', 'description': 'Blob Storage for training data', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/datastores/blob_training_data', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/formacionvm/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f0dc5869990>, 'credentials': {'type': 'account_key'}, 'container_name': 'training-data', 'account_name': 'ntgysapc', 'endpoint': 'core.windows.net', 'protocol': 'https'})

Listar los almacenes de datos nuevamente para verificar que se haya creado un nuevo almacén de datos llamado `blob_training_data`:

In [16]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

blob_training_data
azureml_globaldatasets
workspacefilestore
workspaceartifactstore
workspaceworkingdirectory
workspaceblobstore


## Crear data asset

- `URI_FILE` apunta a un archivo específico.
- `URI_FOLDER` apunta a una carpeta específica.
- `MLTABLE` señala un archivo MLTable que especifica cómo leer uno o más archivos dentro de una carpeta.


In [19]:
# URI_FILE

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './data/diabetes.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Activo de datos que apunta a un archivo local, cargado automáticamente en el almacén de datos predeterminado",
    name="diabetes-local"
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4/datastores/workspaceblobstore/paths/LocalUpload/d52d15a0d7d1e95b90a03f146099424a/diabetes.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-local', 'description': 'Activo de datos que apunta a un archivo local, cargado automáticamente en el almacén de datos predeterminado', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/data/diabetes-local/versions/3', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/formacionvm/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs', 'creation_context': <azure.a

In [20]:
# URI_FOLDER

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

datastore_path = 'azureml://datastores/blob_training_data/paths/data-asset-path/'

my_data = Data(
    path=datastore_path,
    type=AssetTypes.URI_FOLDER,
    description="Activo de datos que apunta a la carpeta de datos de datos en el almacén de datos",
    name="diabetes-datastore-path"
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4/datastores/blob_training_data/paths/data-asset-path/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-datastore-path', 'description': 'Activo de datos que apunta a la carpeta de datos de datos en el almacén de datos', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/data/diabetes-datastore-path/versions/3', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/formacionvm/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0

In [21]:
# MLTABLE
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_path = 'data/'

my_data = Data(
    path=local_path,
    type=AssetTypes.MLTABLE,
    description="MLTable pointing to diabetes.csv in data folder",
    name="diabetes-table"
)

ml_client.data.create_or_update(my_data)

Uploading data (0.52 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 517896/517896 [00:00<00:00, 8791431.48it/s]


Data({'path': 'azureml://subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4/datastores/workspaceblobstore/paths/LocalUpload/58ac422dbe7b4d059d19b1069048a7ad/data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['./diabetes.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-table', 'description': 'MLTable pointing to diabetes.csv in data folder', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/data/diabetes-table/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/formacionvm/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f0da

Para verificar que se hayan creado los nuevos activos de datos, se puede enumerar todos los activos de datos en el espacio de trabajo nuevamente:

In [ ]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

## Leer datos en notebook

- Instalar `mltable`, luego convertir el activo de datos en un dataframe y visualizarlo.

In [22]:
import mltable

registered_data_asset = ml_client.data.get(name='diabetes-table', version=1)
tbl = mltable.load(f"azureml:/{registered_data_asset.id}")
df = tbl.to_pandas_dataframe()
df.head(5)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to i

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,False
1,1147438,8,92,93,47,36,21.240576,0.158365,23,False
2,1640031,7,115,47,52,35,41.511523,0.079019,23,False
3,1883350,9,103,78,25,304,29.582192,1.282870,43,True
4,1424119,1,85,59,27,35,42.604536,0.549542,22,False


## Usar datos en un job

In [23]:
import os

# Crea una carpeta para los archivos de script
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [24]:
%%writefile $script_folder/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

def main(args):
    # read data
    df = get_data(args.input_data)

    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Writing src/move-data.py


Enviar un job que ejecute el script **move-data.py**

In [26]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command

# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-local:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/blob_training_data/paths/datastore-path")
}

# configure job
job = command(
    code="./src",
    command="python move-data.py --input_data ${{inputs.local_data}} --output_datastore ${{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="formacionvm",
    display_name="move-diabetes-data",
    experiment_name="move-diabetes-data"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Monitor your job at https://ml.azure.com/runs/tough_snail_9gjh8fkz59?wsid=/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4&tid=8b85184d-f6f0-4ace-80dd-c3bda5a6c156
